In [4]:
from fastcoref import spacy_component
import spacy

nlp = spacy.load("en_core_web_sm", exclude=["parser", "lemmatizer", "ner", "textcat"])
nlp.add_pipe("fastcoref")

Exception ignored in: <function tqdm.__del__ at 0x7fee26707520>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/tqdm/std.py", line 1161, in __del__
    def __del__(self):
KeyboardInterrupt: 
11/03/2022 08:49:39 - INFO - 	 missing_keys: []
11/03/2022 08:49:39 - INFO - 	 unexpected_keys: []
11/03/2022 08:49:39 - INFO - 	 mismatched_keys: []
11/03/2022 08:49:39 - INFO - 	 error_msgs: []
11/03/2022 08:49:39 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


In [45]:
import logging

logger = logging.getLogger('bgg_predict')
logger.handlers.clear()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

logger.debug('test')

2022-11-03 15:13:02,548 bgg_predict  DEBUG    test
11/03/2022 15:13:02 - DEBUG - 	 test


In [46]:
import re
from typing import List
from dataclasses import dataclass

regex_mail = re.compile(r'\w+(?:\.\w+)*?@\w+(?:\.\w+)+')

@dataclass
class Sentence:
    content: str
    start: int
    end: int
    
    def does_include_pos(self, pos: int) -> bool:
        return self.start <= pos <= self.end

def clean_text(text: str) -> str:
    return regex_mail.sub('', text)

def get_sentences_from_text(text: str) -> List[Sentence]:
    # assert no continuous dots because of text cleared while building the dataset
    sentences = text.split('.')
    
    res = []
    char_accumulator = 0
    for sentence in sentences:
        res.append(Sentence(sentence, char_accumulator, char_accumulator + len(sentence) - 1))
        char_accumulator += len(sentence) + 1
        
    if res[-1].content == '':
        res.pop()
        
    return res

In [47]:
from bisect import bisect_left
from typing import List, Tuple
from dataclasses import dataclass

# necessary to use bisect_left with ranges
@dataclass
class Interval:
    start: int
    end: int
    
    def __lt__(self, other) -> bool:
       return self.start < self.end < other.start
    
    def __eq__(self, other) -> bool:
       return self.start <= other.start <= self.end

def get_sentences_from_clusters(clusters: List[List[Tuple[int, int]]], sentences: List[Sentence]) -> List[List[int]]:
    '''find the sentence each cluster belongs to'''
    sentence_clusters = []
    for cluster in clusters:
        sentence_clusters.append([bisect_left(sentences, Interval(entity[0], entity[1]), key=lambda x: Interval(x.start, x.end)) for entity in cluster])

    return sentence_clusters

text = 'Alice goes down the rabbit hole. Where she would discover a new reality beyond her expectations.'
sentences = get_sentences_from_text(text)
clusters = [[(0, 5), (39, 42), (79, 82)]]
get_sentences_from_clusters(clusters, sentences)

[[0, 1, 1]]

In [48]:
from typing import List
from itertools import groupby
from operator import itemgetter
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

def get_rule_groups_from_sentence_clusters(sentences: List[Sentence], sentence_clusters: List[List[int]]) -> List[List[int]]:
    def normalize_group(group: Set[int]) -> List[List[int]]:
        '''each group could contain multiple consecutive sublists. this method split these sublists'''
        res = []

        # https://stackoverflow.com/a/23861347/5587393
        for k, g in groupby(enumerate(sorted(list(group))), lambda x: x[0] - x[1]):
            res.append(list(map(itemgetter(1), g)))

        return res
    # the graph is built as a directed sparse graph where the first element of each cluster
    # is connected to the other elements in the same cluster
    graph = [[0 for _ in range(len(sentences))] for __ in range(len(sentences))]
    for cluster in sentence_clusters:
        for sentence in cluster[1:]:
            graph[cluster[0]][sentence] = 1

    # find the connected components of the graph created from the clusters returned after coref     
    graph = csr_matrix(graph)
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    groups = [set() for _ in range(n_components)]
    for i, label in enumerate(labels):
        groups[label].add(i)

    return [norm_group for group in groups for norm_group in normalize_group(group)]

In [56]:
text = "Corrections or constructive criticisms?     Email: delta1119@earthlink.net Michael Weston, 2004 Elfenland Quick Reference Setup 1   Each player chooses a color.Place the Boot on Elvenhold and 1 marker in each other city on the board.2   Shuffle the cards.Deal 8 to each player.3   Place the 4 Round # cards on the board in order with the  1  on top.4   Mix tiles face down.Turn 5 tiles face up.5   Each player gets a Summary card and an Obstacle tile.6   Choose a start player and give them the Start Player card Turn Overview  - each phase starts with the Start Player and goes clockwise.1) Refill everyone s hand back up to 8 cards.2) Each player draws a face-down tile.It is kept secret from other players.3) Each player draws either a face-down or face-up tile, and places it face-up in front of them.If a face-up tile was drawn, replace it.Repeat this until each player has 3 face-up tiles.4) Plan routes: Each player places 1 tile on any road on the board, as long as the type of transport is legal for that terrain.An Obstacle tile may only be placed on a road that already has a Transportation tile.A player may pass on one turn and still play at a later turn.This phase ends once all players have passed consecutively.5) Movement: Each player moves his Boot along as many roads as he wishes and has cards for.The Summary card shows how many cards each transport requires to travel in each terrain type.An Obstacle tile requires 1 additional card of the same type in order for the road to be passed.A player takes his color marker off the board as he goes into each city.Rivers: 1 Raft card is required going downstream; 2 are required for going upstream Ferries: 2 Raft cards are required to follow any of the white dotted water routes on both lakes.Caravans: For any non-water travel, a player may play any 3 cards instead of the specifically required type of cards.If an Obstacle is on the road, then 4 cards are required.6) End of round: Change the Round # card to the next number.Pass the Start Player card to the left.Each player keeps 1 tile and 4 cards, discarding any excess.The tile kept may be left face-down if it was already.Shuffle all used and discarded tiles and cards back into supply.Obstacle tiles are removed from the game.Game End The game ends after the 4th round, or if a player collects all 20 of his markers before the end of the 3rd round.The player who collected the most markers wins.If tied, the player holding the most cards in his hand wins.Variants   use in any combination desired.Town Cards: Each player gets 1 Town card face-down.The goal is to end the game in this town.The player s score is reduced by the number of spaces they are away from it at the end of the game.Shorter game: End the game after the 3rd round.Hand Management: in Phase 1, instead of only refilling up to 8 cards, each player is dealt 8 cards.Players must still discard down to 4 cards at the end of the turn (original Alan Moon rule).Ferries: Mare Magnum ferries require 2 Raft cards (normal rule), while Mare Nebulae ferries only require 1 Raft card.Also, players may use a Caravan (any 3 cards) to cross Mare Nebulae (original Alan Moon rules)."    

def get_rules(text: str) -> List[List[int]]:
    text = clean_text(text)
    sentences = get_sentences_from_text(text)
    
    doc = nlp(text, component_cfg = { "fastcoref": {'resolve_text': True} })
    coref_clusters = doc._.coref_clusters
    logger.debug(coref_clusters)
    
    sentence_clusters = get_sentences_from_clusters(coref_clusters, sentences)
    rule_groups = get_rule_groups_from_sentence_clusters(sentences, sentence_clusters)
    
    return ['. '.join([sentences[s_index].content for s_index in group]) for group in rule_groups]
    
get_rules(text)

11/03/2022 15:24:52 - INFO - 	 Tokenize 1 texts...


  0%|          | 0/1 [00:00<?, ?ba/s]

11/03/2022 15:24:53 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-03 15:24:54,374 bgg_predict  DEBUG    [[(200, 211), (287, 296), (929, 938), (1520, 1529)], [(439, 453), (463, 467), (530, 546)], [(632, 648), (649, 651)], [(714, 741), (754, 756)], [(689, 700), (777, 781), (840, 851)], [(785, 799), (819, 821)], [(917, 938), (986, 998)], [(1217, 1228), (1235, 1238), (1267, 1269)], [(392, 406), (1295, 1311)], [(1484, 1492), (1499, 1502), (1533, 1535)], [(468, 489), (1975, 1996)], [(2027, 2033), (2069, 2082), (2108, 2110)], [(2219, 2227), (2237, 2245), (2569, 2577), (2681, 2689), (2708, 2716)], [(2278, 2286), (2306, 2309), (2350, 2391)], [(2406, 2451), (2443, 2446)], [(2511, 2522), (2591, 2603), (2645, 2649)], [(2581, 2590), (2664, 2666)], [(2336, 2349), (2723, 2736)], [(2999, 3011), (3100, 3112)], [(2912, 2921), (3123, 3132)]]
11/03/2022 15:24:54 - DEBUG - 	 [[(200, 211), (287, 296), (929, 938), (1520, 1529)], [(439, 453), (463, 467), (530, 546)], [(632, 648), (649, 651)], [(714, 741), (754, 756)], [(689, 700), (777, 781), (840, 851)], [(785, 799

['Corrections or constructive criticisms?     Email:  Michael Weston, 2004 Elfenland Quick Reference Setup 1   Each player chooses a color',
 'Place the Boot on Elvenhold and 1 marker in each other city on the board',
 '3   Place the 4 Round # cards on the board in order with the  1  on top',
 '4) Plan routes: Each player places 1 tile on any road on the board, as long as the type of transport is legal for that terrain',
 'A player takes his color marker off the board as he goes into each city',
 '2   Shuffle the cards',
 'Deal 8 to each player',
 '4   Mix tiles face down',
 'Turn 5 tiles face up',
 '5   Each player gets a Summary card and an Obstacle tile',
 'The Summary card shows how many cards each transport requires to travel in each terrain type',
 '6   Choose a start player and give them the Start Player card Turn Overview  - each phase starts with the Start Player and goes clockwise',
 'Pass the Start Player card to the left',
 '1) Refill everyone s hand back up to 8 cards',
 '